In [ ]:
# reference:https://blog.prototypr.io/interactive-maps-with-python-part-1-aa1563dbe5a9

In [3]:
import pandas as pd
import geopandas
import numpy as np
# import folium
# import osmnx

In [84]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

marker=folium.CircleMarker(location=[40.738, -73.98],fill=True)
marker.add_to(folium_map)
folium_map

In [85]:
bike_data=pd.read_csv("JC-201907-citibike-tripdata.csv")
bike_data["starttime"]=pd.to_datetime(bike_data["starttime"])
bike_data["stoptime"]=pd.to_datetime(bike_data["stoptime"])
bike_data["hour"]=bike_data["starttime"].map(lambda x: x.hour)

In [86]:
bike_data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,hour
0,1793,2019-07-01 00:00:36.615,2019-07-01 00:30:30.594,3186,Grove St PATH,40.719586,-74.043117,3198,Heights Elevator,40.748716,-74.040443,29595,Subscriber,1978,1,0
1,434,2019-07-01 00:02:54.615,2019-07-01 00:10:09.037,3206,Hilltop,40.731169,-74.057574,3280,Astor Place,40.719282,-74.071262,29647,Subscriber,1988,1,0
2,251,2019-07-01 00:03:27.567,2019-07-01 00:07:38.608,3186,Grove St PATH,40.719586,-74.043117,3205,JC Medical Center,40.716540,-74.049638,26271,Subscriber,1986,1,0
3,636,2019-07-01 00:03:56.169,2019-07-01 00:14:32.418,3206,Hilltop,40.731169,-74.057574,3281,Leonard Gordon Park,40.745910,-74.057271,26225,Customer,1996,2,0
4,623,2019-07-01 00:04:16.206,2019-07-01 00:14:40.050,3206,Hilltop,40.731169,-74.057574,3281,Leonard Gordon Park,40.745910,-74.057271,29230,Customer,1989,1,0


In [87]:
locations=bike_data.groupby("start station id").first()
locations = locations.loc[:, ["start station latitude",
                              "start station longitude",
                              "start station name"]]

In [102]:
selected_hour=7
subset=bike_data[bike_data["hour"]==selected_hour]
departure_counts=subset.groupby("start station id").count()
departure_counts=departure_counts.iloc[:,[0]]
departure_counts.columns=["departure count"]


In [103]:
departure_counts.head()

,departure count
start station id,
3184,25
3185,14
3186,73
3187,51
3191,6


In [104]:
arrival_counts=subset.groupby("end station id").count()
arrival_counts=arrival_counts.iloc[:,[0]]
arrival_counts.columns=["arrival count"]

In [105]:
trip_counts=departure_counts.join(locations).join(arrival_counts)

In [106]:
trip_counts.head()

,departure count,start station latitude,start station longitude,start station name,arrival count
start station id,,,,,
3184,25,40.714145,-74.033552,Paulus Hook,161.0
3185,14,40.717733,-74.043845,City Hall,75.0
3186,73,40.719586,-74.043117,Grove St PATH,992.0
3187,51,40.721124,-74.038051,Warren St,30.0
3191,6,40.718211,-74.083639,Union St,2.0


In [107]:
for index, row in trip_counts.iterrows():
    net_departures=(row["departure count"]-row["arrival count"])
    radius=net_departures/20
    if net_departures>0:
        color="#E3722"
    else:
        color="#0A8A9F"
    folium.CircleMarker(location=(row["start station latitude"],row["start station longitude"]),radius=radius,color=color,fill=True).add_to(folium_map)


In [108]:
folium_map